In [0]:
%run ./0.Config

In [0]:
def send_vendor_rfq_result_emails(_: str) -> str:
    """
    This tool analyzes vendor RFQ submissions, shortlists the top 10 vendors per route based on cost, generates personalized emails using an LLM, and sends the emails to inform vendors whether they were shortlisted or not fro further processes.
    """

    # ------------------ Load CSVs ------------------
    submission_df = (spark.read
        .option("header", True)
        .option("inferSchema", True)
        .csv("abfss://vendor-rfq@genaiautomationsa.dfs.core.windows.net/vendor_response/Vendor_Response.csv")
        .toPandas())

    tracker_df = spark.read.format("delta").load("abfss://vendor-rfq@genaiautomationsa.dfs.core.windows.net/Trackers/clarification_email_tracker").toPandas()

    # ------------------ Date Parsing ------------------
    submission_df["Submitted At"] = pd.to_datetime(submission_df["Submitted At"], dayfirst=True, errors="coerce")
    submission_df = submission_df.rename(columns={"Vendor Email": "Vendor_Email", "Route ID": "Route_ID"})
    tracker_df["email_sent_at"] = pd.to_datetime(tracker_df["email_sent_at"], dayfirst=True, errors="coerce")

    # ------------------ Merge and Filter ------------------
    merged_df = pd.merge(
        submission_df,
        tracker_df[["Vendor_Email", "Route_ID", "email_sent_at"]],
        on=["Vendor_Email", "Route_ID"],
        how="left")

    df_filtered = merged_df[
        (merged_df["email_sent_at"].isna()) |
        (merged_df["Submitted At"] > merged_df["email_sent_at"])]

    df_filtered = df_filtered[submission_df.columns]
    df_filtered = df_filtered.rename(columns={"Vendor_Email": "Vendor Email", "Route_ID": "Route ID"})

    # ------------------ TOTAL COST CALCULATION ------------------
    agg_df = (
        df_filtered.groupby(["Route ID", "Vendor Name", "Vendor Email"])["Total Cost"]
        .sum()
        .reset_index()
        .rename(columns={"Total Cost": "Total Quoted Cost"}))

    # ------------------ TOP 10 VENDORS PER ROUTE ------------------
    top_vendors = (
        agg_df
        .sort_values(["Route ID", "Total Quoted Cost", "Vendor Name", "Vendor Email"])
        .groupby("Route ID")
        .head(10)
        .reset_index(drop=True))

    top_vendors["selected"] = True

    merged_selection = df_filtered.merge(
        top_vendors[["Route ID", "Vendor Name", "Vendor Email", "selected"]],
        on=["Route ID", "Vendor Name", "Vendor Email"],
        how="left"
    )
    merged_selection["selected"] = merged_selection["selected"].fillna(False)

    vendor_group = merged_selection.groupby(["Vendor Name", "Vendor Email", "selected"]).agg({
        "Route ID": lambda x: sorted(set(x))
    }).reset_index()

    # ------------------ LLM SETUP ------------------
    llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

    email_prompt = PromptTemplate(
        input_variables=["vendor_name", "status", "routes"],
        template="""
You are a professional Procurement Manager replying to a vendor about their RFQ submission.

Vendor Name: {vendor_name}
Status: {status}  # "Shortlisted" or "Not Selected"
Routes: {routes}
If Shortlisted:
- Congratulate them and list the routes where they have been selected for further process.
- Send them this link to resubmit the discounted price https://genai-automation-5fciora2kjgdcowb3v9ymk.streamlit.app/.
- Politely request possible discounts for further evaluation and specify the deadline for it as 2 days.

If Not Selected:
- Thank them for their submission.
- Politely inform them they weren’t selected for any routes due to pricing.
- Encourage them to participate in future RFQs.

Sign off with:
Best regards,  
Coca Cola Pvt Ltd  
procurement@cocacola.com
""")

    # ------------------ GENERATE EMAILS ------------------
    emails = []
    unique_vendors = vendor_group.groupby(["Vendor Name", "Vendor Email"])

    for (vendor_name, vendor_email), group in unique_vendors:
        selected_routes = group[group["selected"] == True]["Route ID"].sum()
        selected_routes = sorted(set(selected_routes))

        if selected_routes:
            status = "Shortlisted"
            routes_str = ", ".join(selected_routes)
        else:
            status = "Not Selected"
            routes_str = "None"

        prompt = email_prompt.format(
            vendor_name=vendor_name,
            status=status,
            routes=routes_str)

        content = llm.invoke(prompt).content

        emails.append({
            "to": vendor_email,
            "vendor": vendor_name,
            "status": status,
            "email_content": content})

    # ------------------ SEND EMAILS ------------------
    yag = yagmail.SMTP(EMAIL_USER, EMAIL_PASSWORD)
    for email in emails:
        yag.send(
            to=email["to"],
            subject=f"RFQ Result Notification",
            contents=email["email_content"]
        )
        print(f"✅ Email for '{email['vendor']}' ({email['status']})")

    print(f"📨 Sent {len(emails)} emails.")

# Wrap as single-input Tool
negotiation_tool = Tool(
    name="send_vendor_rfq_result_emails",
    func=send_vendor_rfq_result_emails,
    description="sends the emails to inform vendors whether they were shortlisted or not for further processes",
    return_direct=True
)

In [0]:
tools = [negotiation_tool]

agent = initialize_agent(
    tools=tools,
    llm=ChatDatabricks(endpoint="databricks-llama-4-maverick"),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("sends the emails to inform vendors whether they were shortlisted or not for further processes")